In [2]:
import os.path

import tensorflow as tf
import math
from tqdm.autonotebook import  tqdm
import numpy as np
from scipy import linalg
from Antispoofing.SIWAntispoof.siw_antispoof_helper import get_siw_stratified_name_list_func, get_siw_protocol_frame_dic
from Antispoofing.AntispoofHelpers.dataset_helper import get_test_generator, get_antispoof_frame
from Antispoofing.SIWAntispoof.siw_antispoof_helper import get_siw_train_frame_func
import pandas as pd
from Antispoofing.AntispoofHelpers.hyper_perameter_helper import combine_with_augmentation


/home/jarred/anaconda3/envs/Orfao_Masters/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_1561637/384288019.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import  tqdm


In [3]:
inception_model = tf.keras.applications.InceptionV3(include_top=False,
                              weights="imagenet",
                              pooling='avg')

2022-10-29 17:23:31.472900: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-29 17:23:31.473095: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-29 17:23:31.476601: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-29 17:23:31.476746: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-29 17:23:31.476875: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [24]:
BATCH_SIZE = 32



In [4]:

def compute_embeddings(data_loader_1, data_loader_2):
    def _compute_embeddings(dataloader):
        # for _ in tqdm(range(count)):
        #     images = next(iter(dataloader))
        step_size = dataloader.n // dataloader.batch_size
        embeddings = inception_model.predict(dataloader, step_size, verbose=1)

            # image_embeddings.extend(embeddings)

        # return np.array(image_embeddings)
        return embeddings


    # compute embeddings for real images
    data_loader_1_embeddings = _compute_embeddings(data_loader_1)

    # compute embeddings for generated images
    data_loader_2_embeddings = _compute_embeddings(data_loader_2)

    print(data_loader_1_embeddings.shape," ", data_loader_2_embeddings.shape)
    return data_loader_1_embeddings, data_loader_2_embeddings

In [5]:

def calculate_fid(real_embeddings, generated_embeddings):
    # calculate mean and covariance statistics
    mu1, sigma1 = real_embeddings.mean(axis=0), np.cov(real_embeddings, rowvar=False)
    mu2, sigma2 = generated_embeddings.mean(axis=0), np.cov(generated_embeddings,  rowvar=False)
    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = linalg.sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)

    return fid



In [6]:



def calculate_set_distance(set_1_df_file_path_frame, set_2_df_file_path_frame):

    if set_1_df_file_path_frame.shape[0]> set_2_df_file_path_frame.shape[0]:
        set_1_df_file_path_frame = set_1_df_file_path_frame[:set_2_df_file_path_frame.shape[0]]
    elif set_1_df_file_path_frame.shape[0]< set_2_df_file_path_frame.shape[0]:
        set_2_df_file_path_frame = set_2_df_file_path_frame[:set_1_df_file_path_frame.shape[0]]

    set_1_generator = get_test_generator(set_1_df_file_path_frame)
    set_2_generator = get_test_generator(set_2_df_file_path_frame)

    set_1_embeddings, set_2_embeddings = compute_embeddings(set_1_generator, set_2_generator)

    return calculate_fid(set_1_embeddings, set_2_embeddings)

In [9]:
test_subject_number = 90
ATTACK_TYPE_COMBINATIONS = [
     'A@ASUS-IP7P-IPP2017,A@N',
     'A@ASUS-IP7P-SGS8,A@N',
     'A@ASUS-IPP2017-SGS8,A@N',
     'A@IP7P-IPP2017-SGS8,A@N',
     'A@P,A@N',
     'A@R,A@N',
]

dataset_root = "/home/jarred/Documents/Datasets/SIW"

dataset_csv_name = "siw.csv"

In [44]:
spoof_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {75} and ground_truth == 'spoof'")
spoof_df.head()

real_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {75} and ground_truth == 'real'")
real_df.head()
fid = calculate_set_distance(real_df, spoof_df)
print(round(fid, 2))

,usage_type,ground_truth,subject_number,video_name,frames_present,video_frames,attack_category,directory_path,sensor_id,sensor_name,type_id,type_name,medium_id,medium_name,session_id,session_name
2037,test,real,75,075-1-1-1-1.mov,718,718.0,N,test/real/75/075-1-1-1-1,1,Canon_EOS_T6,1,Live,1,NLV,1,MBF
2038,test,real,75,075-2-1-2-2.mov,1386,1386.0,N,test/real/75/075-2-1-2-2,2,Logitech_C920_Webcam,1,Live,2,ELV,2,YARFEC
2039,test,real,75,075-2-1-1-1.mov,1487,1487.0,N,test/real/75/075-2-1-1-1,2,Logitech_C920_Webcam,1,Live,1,NLV,1,MBF
2040,test,real,75,075-2-1-1-2.mov,1582,1582.0,N,test/real/75/075-2-1-1-2,2,Logitech_C920_Webcam,1,Live,1,NLV,2,YARFEC
2041,test,real,75,075-1-1-2-1.mov,864,864.0,N,test/real/75/075-1-1-2-1,1,Canon_EOS_T6,1,Live,2,ELV,1,MBF


In [7]:
from Antispoofing.AntispoofHelpers.dataset_helper import get_dataframe_by_attack_category, get_dataframe_by_medium_name


def get_category_frame(frame, categories):
    attack_frames = []
    for attack_type in categories:
        if attack_type == "P" or attack_type == "R":
            attack_frames.append(get_dataframe_by_attack_category(frame, attack_type))
        else:
            attack_frames.append(get_dataframe_by_medium_name(frame, attack_type))

    return pd.concat(attack_frames)


In [66]:
spoof_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {90} and ground_truth == 'spoof'")


real_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {90} and ground_truth == 'real'")



test_real_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {75} and ground_truth == 'real'")

test_spoof_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {75} and ground_truth == 'spoof'")


spoof_df = get_category_frame(spoof_df, [ "P"])
test_spoof_df = get_category_frame(test_spoof_df, [ "ASUS", "IP7P", "IPP2017", "SGS8"])



# spoof_df = get_category_frame(spoof_df, ["ASUS", "IP7P", "IPP2017", "SGS8"])
# spoof_df = get_category_frame(spoof_df, [ "IP7P", "IPP2017", "SGS8"])
# spoof_df = get_category_frame(spoof_df, ["P"])
# spoof_df = get_category_frame(spoof_df, ["ASUS", "IP7P", "IPP2017"])

train_proto = pd.concat([spoof_df, real_df])
test_proto = pd.concat([test_spoof_df, test_real_df])

fid = calculate_set_distance(train_proto, test_proto)
print(round(fid, 2))

Found 10644 validated image filenames belonging to 2 classes.
Found 10644 validated image filenames belonging to 2 classes.
333/333 [==============================] - 34s 101ms/step
(10644, 2048)   (10644, 2048)
205196.95


In [63]:
spoof_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {90} and ground_truth == 'spoof'")

In [64]:
temp = get_category_frame(spoof_df, ["P"])
temp.head(30)

,usage_type,ground_truth,subject_number,video_name,frames_present,video_frames,attack_category,directory_path,sensor_id,sensor_name,type_id,type_name,medium_id,medium_name,session_id,session_name
1086,test,spoof,90,090-1-2-2-2.mov,391,391.0,P,test/spoof/90/090-1-2-2-2,1,Canon_EOS_T6,2,Print,2,LR,2,MP
1087,test,spoof,90,090-1-2-1-2.mov,663,663.0,P,test/spoof/90/090-1-2-1-2,1,Canon_EOS_T6,2,Print,1,HR,2,MP
1088,test,spoof,90,090-1-2-1-1.mov,466,466.0,P,test/spoof/90/090-1-2-1-1,1,Canon_EOS_T6,2,Print,1,HR,1,GP
1091,test,spoof,90,090-1-2-2-1.mov,405,405.0,P,test/spoof/90/090-1-2-2-1,1,Canon_EOS_T6,2,Print,2,LR,1,GP


In [15]:
train_spoof_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {90} and ground_truth == 'spoof'")


train_real_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {90} and ground_truth == 'real'")

test_real_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {75} and ground_truth == 'real'")

test_spoof_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {75} and ground_truth == 'spoof'")

TRAIN_SPOOF_COMBINATIONS = [
         [  "IP7P", "IPP2017", "SGS8"],
        [ "ASUS",  "IPP2017", "SGS8"],
        [ "ASUS", "IP7P", "SGS8"],
    [ "ASUS", "IP7P", "IPP2017"],

    [ "ASUS", "IP7P", "IPP2017", "SGS8"],
    [ "P"],
]
TEST_SPOOF_COMBINATIONS = [
    [ "ASUS"],
    [ "IP7P"],
    [ "IPP2017"],
    [ "SGS8"],
    [ "P"],
    [ "ASUS", "IP7P", "IPP2017", "SGS8"],
]
for i in range(len(TRAIN_SPOOF_COMBINATIONS)):
    train_spoof_combination = TRAIN_SPOOF_COMBINATIONS[i]
    test_spoof_combination = TEST_SPOOF_COMBINATIONS[i]

    current_train_spoof_df = get_category_frame(train_spoof_df,train_spoof_combination)
    current_test_spoof_df = get_category_frame(test_spoof_df,  test_spoof_combination)

    # combine train spoof and real
    current_train_spoof_df = pd.concat([current_train_spoof_df, train_real_df])
    current_test_spoof_df = pd.concat([current_test_spoof_df, test_real_df])


    current_train_spoof_file_path_frame = get_antispoof_frame(current_train_spoof_df, dataset_root)
    current_test_spoof_file_path_frame = get_antispoof_frame(current_test_spoof_df, dataset_root)

    current_test_spoof_file_path_frame = current_test_spoof_file_path_frame.sample(frac=1).reset_index(drop=True)

    use_last_only = False
    must_remove_normal = True
    must_use_normal_only=False

    aug_root ="/home/jarred/Documents/TraditionalAugmentation/SIW_90"
    aug_csv ="SIW_90.csv"
    # aug_root ="/home/jarred/Documents/Generated/SIW_KF_90"
    # aug_csv ="SIW_KF_90.csv"


    aug_frame = pd.read_csv(os.path.join(aug_root, aug_csv))
    AUG_PERCENTAGES = [0.05,0.1,0.2, 0.30]

    fid_values ={}
    for aug_percentage in AUG_PERCENTAGES:
        current_aug_frame = combine_with_augmentation(train_frame= current_train_spoof_file_path_frame, aug_frame= aug_frame,aug_root= aug_root,categories= train_spoof_combination, aug_percentage=aug_percentage,must_remove_normal= must_remove_normal,must_use_normal_only=must_use_normal_only, stratified_name_list_func=None)

        current_train_aug_spoof_file_path_frame = pd.concat([current_train_spoof_file_path_frame, current_aug_frame])
        current_train_aug_spoof_file_path_frame = current_train_aug_spoof_file_path_frame.sample(frac=1).reset_index(drop=True)
        fid = calculate_set_distance(current_train_aug_spoof_file_path_frame, current_test_spoof_file_path_frame)
        fid_values[aug_percentage] = round(fid, 2)
        print("Aug %: ", aug_percentage, " FID: ", round(fid, 2))


# spoof_df = get_category_frame(spoof_df, ["ASUS", "IP7P", "IPP2017", "SGS8"])
# spoof_df = get_category_frame(spoof_df, [ "IP7P", "IPP2017", "SGS8"])
# spoof_df = get_category_frame(spoof_df, ["P"])
# spoof_df = get_category_frame(spoof_df, ["ASUS", "IP7P", "IPP2017"])


# test_proto = pd.concat([test_spoof_df, test_real_df])



Found 11204 validated image filenames belonging to 2 classes.
Found 11204 validated image filenames belonging to 2 classes.
351/351 [==============================] - 36s 103ms/step
(11204, 2048)   (11204, 2048)
Aug %:  0.05  FID:  200043.27
Found 11827 validated image filenames belonging to 2 classes.
Found 11827 validated image filenames belonging to 2 classes.
370/370 [==============================] - 37s 101ms/step
(11827, 2048)   (11827, 2048)
Aug %:  0.1  FID:  195497.31
Found 13305 validated image filenames belonging to 2 classes.
Found 13305 validated image filenames belonging to 2 classes.
416/416 [==============================] - 43s 104ms/step
(13305, 2048)   (13305, 2048)
Aug %:  0.2  FID:  187180.36
Found 15147 validated image filenames belonging to 2 classes.
Found 15147 validated image filenames belonging to 2 classes.
474/474 [==============================] - 50s 105ms/step
(15147, 2048)   (15147, 2048)
Aug %:  0.3  FID:  180412.16


In [19]:
train_spoof_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {90} and ground_truth == 'spoof'")


train_real_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {90} and ground_truth == 'real'")

current_train_real_file_path_frame = get_antispoof_frame(train_real_df, dataset_root)

#shuffle
current_train_real_file_path_frame = current_train_real_file_path_frame.sample(frac=1).reset_index(drop=True)

# test_real_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {75} and ground_truth == 'real'")
#
# test_spoof_df = pd.read_csv(os.path.join(dataset_root, dataset_csv_name)).query(f"subject_number == {75} and ground_truth == 'spoof'")

TRAIN_SPOOF_COMBINATIONS = [
         [  "IP7P", "IPP2017", "SGS8"],
        [ "ASUS",  "IPP2017", "SGS8"],
        [ "ASUS", "IP7P", "SGS8"],
    [ "ASUS", "IP7P", "IPP2017"],

    [ "ASUS", "IP7P", "IPP2017", "SGS8"],
    [ "P"],
]
TEST_SPOOF_COMBINATIONS = [
    [ "ASUS"],
    [ "IP7P"],
    [ "IPP2017"],
    [ "SGS8"],
    [ "P"],
    [ "ASUS", "IP7P", "IPP2017", "SGS8"],
]
aug_name = "Trad.csv"

use_last_only = False
must_remove_normal = True
must_use_normal_only=False

# aug_root ="/home/jarred/Documents/Generated/SIW_KF_90"
# aug_csv ="SIW_KF_90.csv"
aug_root ="/home/jarred/Documents/TraditionalAugmentation/SIW_90"
aug_csv ="SIW_90.csv"
aug_frame = pd.read_csv(os.path.join(aug_root, aug_csv))
AUG_PERCENTAGES = [0.05,0.1,0.2, 0.30]
df_dic = {}
for i in range(len(TRAIN_SPOOF_COMBINATIONS)):
    train_spoof_combination = TRAIN_SPOOF_COMBINATIONS[i]


    current_train_spoof_df = get_category_frame(train_spoof_df,train_spoof_combination)


    current_train_spoof_file_path_frame = get_antispoof_frame(current_train_spoof_df, dataset_root)




    fid_intra_values ={}
    for aug_percentage in AUG_PERCENTAGES:
        current_aug_frame = combine_with_augmentation(train_frame= current_train_spoof_file_path_frame, aug_frame= aug_frame,aug_root= aug_root,categories= train_spoof_combination, aug_percentage=aug_percentage,must_remove_normal= must_remove_normal,must_use_normal_only=must_use_normal_only, stratified_name_list_func=None)

        current_train_aug_spoof_file_path_frame = pd.concat([current_train_spoof_file_path_frame, current_aug_frame])
        current_train_aug_spoof_file_path_frame = current_train_aug_spoof_file_path_frame.sample(frac=1).reset_index(drop=True)
        fid = calculate_set_distance(current_train_aug_spoof_file_path_frame, current_train_real_file_path_frame)
        fid_intra_values[aug_percentage] = round(fid, 2)
        print("Aug %: ", aug_percentage, " FID: ", round(fid, 2))

    protocol_name = ""
    if len(TEST_SPOOF_COMBINATIONS[i]) > 1:
        protocol_name = "R"
    else:
        protocol_name = TEST_SPOOF_COMBINATIONS[i][0]

    df_dic[protocol_name] = fid_intra_values
print(df_dic)
df = pd.DataFrame.from_dict(df_dic, orient='index')
df.to_csv(aug_name)



# spoof_df = get_category_frame(spoof_df, ["ASUS", "IP7P", "IPP2017", "SGS8"])
# spoof_df = get_category_frame(spoof_df, [ "IP7P", "IPP2017", "SGS8"])
# spoof_df = get_category_frame(spoof_df, ["P"])
# spoof_df = get_category_frame(spoof_df, ["ASUS", "IP7P", "IPP2017"])


# test_proto = pd.concat([test_spoof_df, test_real_df])



Found 4736 validated image filenames belonging to 1 classes.
Found 4736 validated image filenames belonging to 1 classes.
148/148 [==============================] - 18s 120ms/step
(4736, 2048)   (4736, 2048)
Aug %:  0.05  FID:  42658.06
Found 5000 validated image filenames belonging to 1 classes.
Found 5000 validated image filenames belonging to 1 classes.
157/157 [==============================] - 19s 119ms/step
(5000, 2048)   (5000, 2048)
Aug %:  0.1  FID:  43594.0
Found 5624 validated image filenames belonging to 1 classes.
Found 5624 validated image filenames belonging to 1 classes.
176/176 [==============================] - 21s 118ms/step
(5624, 2048)   (5624, 2048)
Aug %:  0.2  FID:  45688.5
Found 6428 validated image filenames belonging to 1 classes.
Found 6428 validated image filenames belonging to 1 classes.
201/201 [==============================] - 24s 120ms/step
(6428, 2048)   (6428, 2048)
Aug %:  0.3  FID:  47888.4
Found 4728 validated image filenames belonging to 1 classe